# Topic 1. Problem Solving by Search

Implementation of search algorithms and search problems for AIMA.

We start by defining the abstract class for a `Problem`; problem domains will subclass this, and then you can create individual problems with specific initial states and goals. We also define a `Node` in a search tree, and some functions on nodes: `expand` to generate successors, and `path_actions`, `path_states` and `path` to recover aspects of the path from the node.  Finally, a `PriorityQueue`, which allows you to keep a collection of items, and continually remove from it the item with minimum `f(item)` score.\

### Example 1-1: The Route-Finding Domain

Like all state-space search problems, in a route-finding problem you will be given:
- A start state (for example, `'A'` for the city Arad).
- A goal state (for example, `'B'` for the city Bucharest).
- Actions that can change state (for example, driving from `'A'` to `'S'`).

You will be asked to find:
- A path from the start state, through intermediate states, to the goal state.

We'll use this map:

<img src="images/romania.jpg" height="366" width="603">

A state-space search problem can be represented by a *graph*, where the vertices of the graph are the states of the problem (in this case, cities) and the edges of the graph are the actions (in this case, driving along a road). More description will be shown below.


# 1-1. Import python package & module

In [1]:
import math
import sys

# For some data structure implementation
import heapq
from collections import defaultdict, deque, Counter

# For visialization
import matplotlib.pyplot as plt

# 1-2. Problem abstract class definition

In [2]:
class Problem(object):
    def __init__(self, initial=None, goal=None, **other_keywords):
        """Specify the initial and goal states.
        Subclasses can use other keywords if they want."""
        self.__dict__.update(initial=initial, goal=goal, **other_keywords) 

    def actions(self, state):           raise NotImplementedError
    def result(self, state, action):    raise NotImplementedError
    def is_goal(self, state):           return state == self.goal
    def step_cost(self, s, action, s1): return 1
    def h(self, node):                  return 0

# 1-3. Node definition

In [3]:
class Node:
    '''A Node in a search tree.'''
    def __init__(self, state, parent=None, action=None, path_cost=0):
        # __dict__ store this object's all attributes
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)
    
    '''All Reserve words are not introduced here. If you are interest in them, please Google them'''
    # __repr__ is a built-in function used to compute the '''official''' string reputation of an object.
    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.state < other.state
    
failure = Node('failure', path_cost=float('inf')) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=float('inf')) # Indicates iterative deeepening search was cut off.

def expand(problem, node):
    '''Expand a node, generating the children nodes.'''
    s = node.state
    for action in problem.actions(s): 
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.step_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    '''The sequence of actions to get to this node.'''
    if node.parent is None:
        return []
    else: 
        return path_actions(node.parent) + [node.action]


def path_states(node):
    '''The sequence of states to get to this node.'''
    if node.parent is None:
        return ([] + [node.state])
    else:
        return (path_states(node.parent)) + [node.state]


def path(node):
    '''Alternating states and actions to get to this node.'''
    if node.parent is None:
        return ([] + [node.state])
    else:
        return (path(node.parent) + [node.action] ) + [node.state]

# 1-4. Queue definition
To implement the graph searching algorithm, we must choose a proper data structure to store the **node**. We show the `First-in-first-out queue` and `Last-in-first-out queue` (also famous with **stack**). The animation created by [visualgo.net](https://visualgo.net/en/dfsbfs) shows the process for inserting/removing a node. In practice, we implement the FIFOQueue with `deque` which is the python module from `collection` package and implement LIFOQueue with python `list`

### First-in-first-out (queue)
<img src="images/animated_queue.gif" width="480px" align="left">
<br /><br /><br /><br /><br />

### Last-in-first-out queue (stack)
<img src="images/animated_stack.gif" width="240px" align="left">
<br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br /><br />

### PriorityQueue
In addition, a `PriorityQueue`, which allows you to keep a collection of items, and continually remove from it the item with minimum `f(item)` score.

In [4]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

# 1-5. Search Algorithms

Here are the state-space search algorithms covered in the **AIMA** book:

### Basic search algorithm review

To bring something to your mind, we show two basic search algorithm below. Hope it can help you get the code quickly XD. The animation source is create by [visualgo.net](https://visualgo.net/en/dfsbfs)

|  **BFS** | **DFS**  |
|:---------------------:|:---------------------:|
| <img src="images/animated_bfs.gif" width="300px">  | <img src="images/animated_dfs.gif" width="300px"> |


In [5]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    frontier = FIFOQueue([Node(problem.initial)])
    reached = set()
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


def depth_limited_search(problem, limit=5):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    solution = failure
    while frontier:
        node = frontier.pop()
        if len(node) > limit:
            solution = cutoff
        else:
            for child in expand(problem, node):
                if problem.is_goal(child.state):
                    return child
                frontier.append(child)
    return solution

def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result

In [6]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    reached = {}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure

def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=lambda node: node.path_cost)


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda node: node.path_cost + h(node))

# 1-6. Problem Domains

Now we turn our attention to defining some problem domains.

### The Route-Finding Domain

Like all state-space search problems, in a route-finding problem you will be given:
- A start state (for example, `'A'` for the city Arad).
- A goal state (for example, `'B'` for the city Bucharest).
- Actions that can change state (for example, driving from `'A'` to `'S'`).

You will be asked to find:
- A path from the start state, through intermediate states, to the goal state.

We'll use this map:

<img src="images/romania.jpg" height="366" width="603">

A state-space search problem can be represented by a *graph*, where the vertices of the graph are the states of the problem (in this case, cities) and the edges of the graph are the actions (in this case, driving along a road).

We'll represent a city by its single initial letter. 
We'll represent the graph of connections as a `dict` that maps each city to a list of the neighboring cities (connected by a road). For now we don't explicitly represent the actions, nor the distances
between cities.

## Example 1-1. Route Finding Problems

In [7]:
def sldistance(A, B):
    "Straight-line distance between two 2D points."
    return abs(complex(*A) - complex(*B))

def multimap(pairs):
    "Given (key, val) pairs, make a dict of {key: [val,...]}."
    result = defaultdict(list)
    for key, val in pairs:
        result[key].append(val)
    return result

class Map:
    """A map of places in a 2D world: a graph with vertexes and links between them. 
    `links` can be either [(v1, v2)...] pairs, or {(v1, v2): distance...}.
    If `directed=False` then for every (v1, v2) link, we add a (v2, v1).
    `locations` is optional and can be {v1: (x, y)} 2D locations of vertexes."""
    def __init__(self, links, locations=None, directed=False):
        if not hasattr(links, 'items'): # Make `links` into a dict
            links = defaultdict(lambda: 1, links)
        if not directed:
            for (v1, v2) in list(links):
                links[v2, v1] = links[v1, v2]
        self.distances = links
        self.locations = locations or defaultdict(lambda: (0, 0))
        self.neighbors = multimap(links)


romania = Map(
    {('O', 'Z'): 71, ('O', 'S'): 151, ('A', 'Z'): 75, ('A', 'S'): 140, ('A', 'T'): 118, 
     ('L', 'T'): 111, ('L', 'M'): 70, ('D', 'M'): 75, ('C', 'D'): 120, ('C', 'R'): 146, 
     ('C', 'P'): 138, ('R', 'S'): 80, ('F', 'S'): 99, ('B', 'F'): 211, ('B', 'P'): 101, 
     ('B', 'G'): 90, ('B', 'U'): 85, ('H', 'U'): 98,  ('E', 'H'): 86, ('U', 'V'): 142, 
     ('I', 'V'): 92, ('I', 'N'): 87, ('P', 'R'): 97},
    dict(
    A=(91, 492), B=(400, 327), C=(253, 288), D=(165, 299), E=(562, 293), F=(305, 449),
    G=(375, 270), H=(534, 350), I=(473, 506), L=(165, 379), M=(168, 339), N=(406, 537),
    O=(131, 571), P=(320, 368), R=(233, 410), S=(207, 457), T=(94, 410), U=(456, 350),
    V=(509, 444), Z=(108, 531)))

In [8]:
# Show the neighbors of the place
romania.neighbors['A']

['Z', 'T', 'S']

In [9]:
# Show distance between two adjacent place
romania.distances['A', 'Z']

75

In [10]:
# Show the location infomation of the place
romania.locations['A']

(91, 492)

<img src="images/romania.jpg" height="366" width="603">

In [11]:
class RouteProblem(Problem):
    """A problem to find a route between places on a map.
    Use RouteProblem('S', 'G', map=Map(...)})"""
    
    def actions(self, state): 
        """The places neighboring `state`. (Action names are same as place names.)"""
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """Go to the `action` place, if the map says that is possible."""
        return action if action in self.map.neighbors[state] else state
    
    def step_cost(self, s, action, s1):
        """The actual distance between s and s1."""
        return self.map.distances[s, s1]
    
    def h(self, node):
        "Straight-line distance between state and the goal."
        locs = self.map.locations
        return sldistance(locs[node.state], locs[self.goal])

In [12]:
# Solve a problem (which gives a node) and recover the sequence of states in that node's path
problem_r1 = RouteProblem('A', 'B', map=romania)
# Breadth first search finds a solution with fewer steps, but in this case higher path cost
path_states(breadth_first_search(problem_r1))

['A', 'S', 'F', 'B']

# 1-7. Reporting Metrics

Now let's gather some metrics on how well each algorithm does. We'll use CountCalls to wrap a Problem object in such a way that calls to its methods are delegated, but each call increments a counter. Once we've solved the problem, we print out summary statistics.

In [13]:
class CountCalls:
    """Delegate all attribute accesses to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        self._counts[attr] += 1
        return getattr(self._object, attr)
        
def report(searchers, problems_list):
    "Show metrics for each searcher on each problem."
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems_list:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(len=len(path_actions(soln)), cost=soln.path_cost)
            total_counts += counts
            report_line(counts, type(p).__name__)
        report_line(total_counts, 'TOTAL\n')
        
def report_line(counts, name):
    "Print one line of the report."
    print('{:9,d} nodes explored |{:7,d} goal |{:5.0f} cost |{:3d} steps | {}'
          .format(counts['result'], counts['is_goal'], 
                  counts['cost'], counts['len'], name))

In [14]:
r1 = RouteProblem('A', 'B', map=romania)
r2 = RouteProblem('N', 'L', map=romania)

# Here's a tiny report
report([breadth_first_search, astar_search], [r1, r2])

breadth_first_search:
       31 nodes explored |     13 goal |  450 cost |  3 steps | RouteProblem
       45 nodes explored |     21 goal | 1085 cost |  9 steps | RouteProblem
       76 nodes explored |     34 goal | 1535 cost | 12 steps | TOTAL

astar_search:
       15 nodes explored |      6 goal |  418 cost |  4 steps | RouteProblem
       35 nodes explored |     16 goal |  910 cost |  9 steps | RouteProblem
       50 nodes explored |     22 goal | 1328 cost | 13 steps | TOTAL

